## Imports

In [28]:
from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
from typing import Dict
from IPython.display import display, Markdown
from dotenv import load_dotenv
import requests
import os

In [2]:
load_dotenv(override = True)

True

In [29]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

In [4]:
search_agent = Agent (
    name="Search Agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    tools=[WebSearchTool(search_context_size='low')],       # giving it a websearch tool.
    model_settings=ModelSettings(tool_choice='required')    # making it required to use tools.
)

In [9]:
message = "Latest AI Agent frameworks"
with trace("Web Search Tool Trace"):
    result = await Runner.run(search_agent, message)

In [11]:
display(Markdown(result.final_output))

Recent advancements in AI agent frameworks have introduced several innovative tools designed to enhance the development and deployment of intelligent systems. Notable among these are Project Mariner, Manus, and OpenClaw.

**Project Mariner** is a research prototype by Google DeepMind, launched in December 2024. It automates web-based tasks like online shopping and information retrieval, aiming to boost user productivity. Operating as a Chrome extension, it interprets complex goals and navigates websites to execute tasks, with user oversight. As of May 2025, Project Mariner is available to Google AI Ultra subscribers in the US and is integrated into the Gemini API and Vertex AI. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Project_Mariner?utm_source=openai))

**Manus** is an autonomous AI agent developed by Butterfly Effect Pte. Ltd., launched on March 6, 2025. It performs complex real-world tasks without continuous human guidance, including writing and deploying code. In December 2025, Meta Platforms acquired Manus, valuing the deal between US$2-3 billion. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

**OpenClaw** is a free and open-source autonomous AI assistant software developed by Peter Steinberger, released in November 2025. It executes tasks via large language models, using messaging platforms as its main user interface. OpenClaw gained popularity in late January 2026 and, on February 14, 2026, Steinberger announced joining OpenAI, with the project moving to an open-source foundation. ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenClaw?utm_source=openai))

These frameworks represent significant strides in AI agent development, offering diverse capabilities for automating tasks and enhancing user interaction. 

## Project

We will come up with our own personal web searcher agent as follows...
- **Search Query Generator:** Takes a user's research query and come up with a few different web search queries
- **Report Generator:** Based on the search queries provided, we will write a report on it
- **Text Message:** Text the report to us

### Agent 1: Web Search Query Agent

In [17]:
message = "Latest AI Agent frameworks"

In [12]:
HOW_MANY_SEARCHES = 3

In [13]:
search_instructions = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

Let's define output template (class)

In [15]:
class WebSearchQueryItem(BaseModel):
    reason: str = Field(description="Provide reasoning why this specific web search query item is needed")
    query: str = Field(description="The search term to use for the web search")

class WebSearches(BaseModel):
    web_search_queries: list[WebSearchQueryItem] = Field(description = "List of web search queries as WebSearchQueryItem objects")

In [22]:
web_search_query_agent = Agent(
    name="SearchAgent",
    instructions=search_instructions,
    model="gpt-4o-mini",
    output_type=WebSearches
)

### Agent 2. Report Generator Agent

In [34]:
report_instructions = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query and web searches.\n"
    "You should generate a report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 100-150 words."
)

report_agent = Agent(
    name="Report Agent",
    instructions=report_instructions,
    model="gpt-4o-mini",
    tools=[WebSearchTool(search_context_size='low')],
    model_settings=ModelSettings(tool_choice='required')
)

### Agent 3: Messenger Agent

In [30]:
@function_tool
def send_message(message: str) -> str:
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
messenger_instructions = """
Craft a short message to the user that uses the following...
- 1-2 sentence message introduction
- Generated report from the Report Agent (DO NOT MAKE UP ANYTHING HERE)
Then send the message using your tools.
Output an acknowledgement that the message has been sent as well as the final output from the Report Agent  
"""

In [32]:
message_agent = Agent(
    name="Messenger Agent",
    instructions=messenger_instructions,
    model="gpt-4o-mini",
    tools=[send_message],
    model_settings=ModelSettings(tool_choice='required')
)

### Main Agent

In [35]:
web_search_tool = web_search_query_agent.as_tool(
    tool_name="web_search_tool",
    tool_description="To turn a user's query into multiple search queries"
)

report_tool = report_agent.as_tool(
    tool_name="report_tool",
    tool_description="Uses the provided web search queries to generate a short report to answer user's query"
)

In [36]:
tools = [web_search_tool, report_tool]
handoffs = [message_agent]

In [37]:
instructions = """You are a professional researcher designed to answer a user's query with a report
## Steps
Follow these steps carefully...
1. **Generate Web Search Queries:** Using the web search tool, you will take the user's query and create a few web 
search queries that another agent will use.
2. **Generate a Report:** Using the reporter tool you will perform web searchs using the queries provided to create
a report for the user that answers their original query
3. **Handoff:** Handoff the generated report to the messenger agent (DO NOT PERFORM ANYTHING ELSE AFTERWARDS)

Output confirmation that the message was sent followed by the final report
"""

In [38]:
researcher = Agent(
    name="Researcher",
    instructions=instructions,
    model="gpt-4o-mini",
    tools = tools,
    handoffs = handoffs
)

In [39]:
with trace("Resarch Agentic Workflow"):
    result = await Runner.run(researcher, message)

In [42]:
display(Markdown(result.final_output))

The message regarding the latest AI agent frameworks has been sent successfully. Here’s a summary from the report:

1. **LangChain**: A comprehensive framework designed for LLM applications with extensive integrations.
2. **AutoGen**: Microsoft’s framework for multi-agent collaboration.
3. **CrewAI**: An open-source framework for role-playing AI agents.
4. **AgentScope**: A developer-focused framework for building agentic applications.
5. **LlamaIndex**: A context-aware framework supporting various data types.
6. **OpenAI Swarm**: Focuses on multi-agent coordination for complex tasks.
7. **Microsoft Semantic Kernel**: Simplifies AI model integration into applications.

Let me know if you need more information!